In [1]:
import os
pwd = os.getcwd()

pwd

'/home/trpquo/projekti/data/art'

In [2]:
import pandas as pd
import numpy as np
import sys
from utils.secrets import cloud

if "/home/" in pwd:
    sys.path.insert(0, "./utils")
    sys.path.insert(0, cloud/"../")
elif "\\cloud\\" in pwd:
    sys.path.insert(0, "../utils")


from utils.pipe_tools.workbench import Stats

### Učitavanje podataka iz triju baza

Sve tri baze su unaprijed pripremljene preko `wikiart-setup.py`, `wikiart-emotions-setup.py` i moćnih ručno izrađenih alata za baratanje ovim bazama smještenih u `../../utils` direktoriju.

WikiArt baza je ta koja je pripremljena s već kategoriziranim stilovima. Ta kategorizacija nije nužno idealna i vjerojatno će ju trebati ponoviti, ali na probranom uzorku izložaka jer, čini mi se, nema razloga teretiti cijelu bazu već samo uzorak za trening i testiranje.


In [3]:
wikiart = Stats(data_url="./data/wikiart_paintings.parquet", title="Wikiart")
emotions = Stats(data_url="./data/Wikiart-Emotions-All-t.parquet", title="Wikiart-emotions")
artemis = Stats(data_url="./data/artemis_dataset_release_v0-t2.parquet", title="ArtEmis")

In [14]:
from utils.secrets import cloud

wikiart_artworks = pd.read_csv(cloud/"wikiart/wikidata/wikiart.data")
wikiart_artists = pd.read_csv(cloud/"wikiart/wikidata/labels.data")
wikiart_full.shape, wikiart_artists.shape, wikiart.raw_data.shape

((215967, 6), (5292, 10), (160850, 33))

In [15]:
wikiart_artworks.head()

,contentId,url,style,genre,artistContentId,artistUrl
0,337208,no-great-crime-1983,Street art,graffiti,335710,3d
1,337204,3d-1984,Street art,graffiti,335710,3d
2,337213,wild-bunch-1984,Street art,graffiti,335710,3d
3,337210,serious-art-1986,Street art,graffiti,335710,3d
4,337209,robert-de-niro-taxi-driver-1988,Street art,graffiti,335710,3d


In [19]:
len( wikiart_artworks["style"].unique() ), len( wikiart_artworks["genre"].unique() ), 

(1552, 726)

In [29]:
from utils.styles import all_styles, genre_categories

all_genres = []
for cat in genre_categories.values():
    all_genres += cat

len(all_genres)

68

In [13]:
wikiart_labels.head()

,contentId,url,artistName,lastNameFirst,image,wikipediaUrl,birthDay,deathDay,birthDayAsString,deathDayAsString
0,335710,3d,3D,3D,https://uploads3.wikiart.org/images/3d.jpg!Por...,http://en.wikipedia.org/wiki/Robert_Del_Naja,/Date(-124502400000)/,/Date(253402300799999)/,"January 21, 1966",NaN
1,9223372032559846617,a-r-valgorth,A. R. Valgorth,NaN,https://uploads3.wikiart.org/00273/images//ava...,NaN,/Date(252460800000)/,/Date(253402300799999)/,1978,NaN
2,314332,hans-von-aachen,Hans von Aachen,Aachen Hans von,https://uploads8.wikiart.org/temp/e4d609cf-bbc...,http://en.wikipedia.org/wiki/Hans_von_Aachen,/Date(-13190860800000)/,/Date(-11197353600000)/,1552,"March 4, 1615"
3,9223372032559872115,alvar-aalto,Alvar Aalto,Aalto Alvar,https://uploads1.wikiart.org/00335/images//454...,https://en.wikipedia.org/wiki/Alvar_Aalto,/Date(-2269209600000)/,/Date(200620800000)/,"February 3, 1898","May 11, 1976"
4,316166,vilmos-aba-novak,Vilmos Aba-Novak,Aba-Novak Vilmos,https://uploads8.wikiart.org/images/vilmos-aba...,http://en.wikipedia.org/wiki/Vilmos_Aba-Nov%C3...,/Date(-2391984000000)/,/Date(-891734400000)/,"March 15, 1894","September 29, 1941"


### Spajanje WikiArt baze s Artemis bazom


In [ ]:
mappings = wikiart.raw_data.merge(artemis.raw_data.rename(columns={"emotion":"artemis", "Artist": "artistName", "Title": "title"})[["artistName", "title", "artemis"]], on=["artistName", "title"], how="left")
mappings.dropna(subset=["artemis"]).shape, artemis.raw_data.shape

### Spajanje WikiArt-Artemis baze s WikiArt_emotions bazom


In [ ]:
emotions_cols = emotions.raw_data[['emotion', 'Is painting', 'Face/body', 'Rating', "Title", "Artist", "Year", 'webUrl']].rename(columns={"emotion": "emotions", "Title": "title", "Artist": "artistName", "webUrl": "webUrl2"})
mappings_em = mappings.merge(emotions_cols, on=["artistName", "title"], how="left")

mappings_em.dropna(subset=["emotions"]).shape, emotions_cols.shape

### Čišćenje i optimiziranje


In [ ]:
# from styles import style_categories as styles
int_columns = ['Icon', 'Realism', 'Romanticism', 'Impressionism', 'Expressionism', 'Surrealism', 'Naive Art', 'Fauvism', 'Brutalism', 'Dada', 'Cubism',
    'Abstract Order', 'Abstract Chaos', 'Applied Art', 'Oriental Illustration', 'Photography', 'abstract', 'painting', 'drawing', 'composit', 'applied art', 'photography']
large_int_columns = [ 'contentId', 'artistContentId' ]
mappings_em[ int_columns ] = mappings_em[ int_columns ].astype(np.int8)
mappings_em[ large_int_columns ] = mappings_em[ large_int_columns ].astype(np.int64)

In [ ]:
mappings_em.year = mappings_em.year.apply(pd.to_numeric).astype("Int64")
mappings_em.sample(5)

In [ ]:
mappings_em = mappings_em.set_index("contentId")

In [ ]:
cat_columns = [ "style", "genre", "Is painting", "Face/body"]
mappings_em[ cat_columns ] = mappings_em[ cat_columns ].astype("category")
mappings_em.sample(5)

### Pohranjivanje kompozitne baze


In [ ]:
mappings_em.shape

In [ ]:
mappings_em[['artistName', 'title', 'year', 'style', 'genre', 'photography', 'artemis', 'emotions',
       'Is painting', 'Face/body', 'Rating', 'artistContentId',
       'artistUrl', 'url', 'localUrl', 'webUrl', 'webUrl2']].rename(columns={"url": "paintingUrl"}).to_parquet("./data/wikiart_composed.parquet")